# Option Data

With this notebook, we illustrate available option data.

In [ ]:
import sys
sys.path.append('../python/')

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)  # suppress pandas sql warning

import plotly.graph_objects as go

In [ ]:
import data
data.initialise()

We select a stock (symbol) and a data (in ISO format).

In [ ]:
symbol = 'NVDA'
date = '2022-09-16'

## Quoted Option Prices

In [ ]:
def plot_smile_data_for_expiry(symbol, date, expiry):
    spot = data.spot_price(symbol, date)['close'].values[0]
    df = data.smile_prices(symbol, date, expiry)
    strikes = df.columns
    df = df.reset_index()
    df_c = df[ df['call_put'] == 'Call']
    df_p = df[ df['call_put'] == 'Put']
    #
    c_mid = 0.5 * (df_c[df_c['data_name'] == 'ask'][strikes].values[0] + df_c[df_c['data_name'] == 'bid'][strikes].values[0])
    p_mid = 0.5 * (df_p[df_p['data_name'] == 'ask'][strikes].values[0] + df_p[df_p['data_name'] == 'bid'][strikes].values[0])
    c_ba = 0.5 * (df_c[df_c['data_name'] == 'ask'][strikes].values[0] - df_c[df_c['data_name'] == 'bid'][strikes].values[0])
    p_ba = 0.5 * (df_p[df_p['data_name'] == 'ask'][strikes].values[0] - df_p[df_p['data_name'] == 'bid'][strikes].values[0])
    #
    title_string = "Option Prices %s, %s, spot %.2f, expiry %s" % (symbol, str(date), spot, str(expiry))
    error_c = dict(type='data', array = c_ba, visible=True)
    error_p = dict(type='data', array = p_ba, visible=True)
    c_trace = go.Scatter(x = strikes, y = c_mid, mode='lines', name='Call', error_y=error_c)
    p_trace = go.Scatter(x = strikes, y = p_mid, mode='lines', name='Put', error_y=error_p)
    layout = go.Layout(title=dict(text=title_string))
    fig = go.Figure(data=[c_trace, p_trace], layout=layout)
    fig.show()
    #

def plot_smile_data(symbol, date):
    expiries = data.smile_expiries(symbol, date)
    for expiry in expiries['expiration']:
        plot_smile_data_for_expiry(symbol, date, expiry)
    #

plot_smile_data(symbol, date)

## Quoted Implied Volatilities

In [ ]:
def plot_smile_vols_for_expiry(symbol, date, expiry):
    spot = data.spot_price(symbol, date)['close'].values[0]
    df = data.smile_prices(symbol, date, expiry, values = ['vol',])
    strikes = df.columns
    df = df.reset_index()
    df_c = df[ df['call_put'] == 'Call']
    df_p = df[ df['call_put'] == 'Put']
    #
    c_vols = df_c[strikes].values[0]
    p_vols = df_p[strikes].values[0]
    #
    title_string = "Option Vols %s, %s, spot %.2f, expiry %s" % (symbol, str(date), spot, str(expiry))
    c_trace = go.Scatter(x = strikes, y = c_vols, mode='lines+markers', name='Call')
    p_trace = go.Scatter(x = strikes, y = p_vols, mode='lines+markers', name='Put')
    layout = go.Layout(title=dict(text=title_string))
    fig = go.Figure(data=[c_trace, p_trace], layout=layout)
    fig.show()
    #

def plot_smile_vols(symbol, date):
    expiries = data.smile_expiries(symbol, date)
    for expiry in expiries['expiration']:
        plot_smile_vols_for_expiry(symbol, date, expiry)
    #

plot_smile_vols(symbol, date)

We see that implied Call and Put volatilities do not match/overlap. This indicates that the forward price was not estimated correctly when calculating impled volatilities.